In [44]:
import torch, tiktoken

In [45]:
tokenizer = tiktoken.get_encoding("gpt2")

In [46]:
with open("thelostrace.txt", "r") as f:
    raw_text = f.read()

raw_text

'Cororuc glanced about him and hastened his pace. He was no coward, but he did not like the place. Tall trees rose all about, their sullen branches shutting out the sunlight. The dim trail led in and out among them, sometimes skirting the edge of a ravine, where Cororuc could gaze down at the tree-tops beneath. Occasionally, through a rift in the forest, he could see away to the forbidding hills that hinted of the ranges much farther to the west, that were the mountains of Cornwall.\n\nIn those mountains the bandit chief, Buruc the Cruel, was supposed to lurk, to descend upon such victims as might pass that way. Cororuc shifted his grip on his spear and quickened his step. His haste was due not only to the menace of the outlaws, but also to the fact that he wished once more to be in his native land. He had been on a secret mission to the wild Cornish tribesmen; and though he had been more or less successful, he was impatient to be out of their inhospitable country. It had been a long, 

In [47]:
enc_text = tokenizer.encode(raw_text)
enc_text[:10]


[10606, 273, 1229, 27846, 546, 683, 290, 19338, 2945, 465]

In [48]:
from torch.utils.data import Dataset, DataLoader

In [52]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt) #tokenizes all of text

        for i in range(0, len(token_ids) - max_length, stride): # uses sliding window to chunk book into sequences of max_length
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))


    def __len__(self): #returns total # of rows in data set
        return len(self.input_ids)
    def __getitem__(self, idx): #returns single row from dataset
        return self.input_ids[idx], self.target_ids[idx]



In [54]:
#data loader for gptdatasetv1
def create_dataloader_v1(txt, batch_size=4, max_length=256,stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2") # initializes tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)#creates data set
    dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last=drop_last,
                            num_workers=num_workers) #drops last batch if its shorter than the specificed batch_size wanted for training
                            #also number of cpu processes for preprocessing
    return dataloader 



In [55]:
# testing data loader
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)




[tensor([[10606,   273,  1229, 27846]]), tensor([[  273,  1229, 27846,   546]])]


In [56]:
#trying another batch

second_batch = next(data_iter)
second_batch


[tensor([[  273,  1229, 27846,   546]]),
 tensor([[ 1229, 27846,   546,   683]])]

In [58]:
#testing other variabls
dataloader = create_dataloader_v1(raw_text, batch_size=5, max_length=2, stride=2, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[10606,   273],
        [ 1229, 27846],
        [  546,   683],
        [  290, 19338],
        [ 2945,   465]]), tensor([[  273,  1229],
        [27846,   546],
        [  683,   290],
        [19338,  2945],
        [  465,  8761]])]


In [62]:
#and messing with stride variable as well as max_length/batch_size and seeing how shuffle true looks vs false

dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=9, stride=3, shuffle=True)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   11,  1626,  1336,  6504,    13,  9170, 14187,  3500,   284],
        [   11,   788,  2900,   290,   925,   572,   351,   257,  6283]]), tensor([[ 1626,  1336,  6504,    13,  9170, 14187,  3500,   284,  7440],
        [  788,  2900,   290,   925,   572,   351,   257,  6283,   427]])]


In [63]:
#messed with different variables to see what stride did with input field changing position but using this setup for this section
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[10606,   273,  1229, 27846],
        [  546,   683,   290, 19338],
        [ 2945,   465,  8761,    13],
        [  679,   373,   645, 26769],
        [   11,   475,   339,   750],
        [  407,   588,   262,  1295],
        [   13, 22676,  7150,  8278],
        [  477,   546,    11,   511]])

Targets:
 tensor([[  273,  1229, 27846,   546],
        [  683,   290, 19338,  2945],
        [  465,  8761,    13,   679],
        [  373,   645, 26769,    11],
        [  475,   339,   750,   407],
        [  588,   262,  1295,    13],
        [22676,  7150,  8278,   477],
        [  546,    11,   511,   424]])


In [69]:

#tried this at first, but it needs inputs to turn into a list before decoding #tokenizer.decode(inputs)
for row in inputs:
    print( tokenizer.decode( row.tolist() ) )


Cororuc glanced
 about him and hast
ened his pace.
 He was no coward
, but he did
 not like the place
. Tall trees rose
 all about, their


In [71]:
#embedding layer
vocab_size = 4
output_dim = 8

embedding_inputs =  torch.nn.Embedding(vocab_size, output_dim)
print(embedding_inputs.weight)


Parameter containing:
tensor([[ 1.2296, -1.1616,  0.0488, -0.0969, -0.9829,  0.9344, -0.6580,  2.3706],
        [ 0.7454, -0.9280,  0.0543,  0.2805,  0.1924, -0.2474, -1.2096, -1.7903],
        [ 2.6335,  0.7094,  1.4464, -0.4417,  0.2587,  1.4437, -0.2084,  0.4695],
        [-0.1834,  2.3721, -0.0296, -1.4014, -0.5290,  1.0046, -0.9236, -0.6688]],
       requires_grad=True)


In [73]:
#get the weights of embedding_inputs
input_weights = embedding_inputs.weight.data
input_weights

tensor([[ 1.2296, -1.1616,  0.0488, -0.0969, -0.9829,  0.9344, -0.6580,  2.3706],
        [ 0.7454, -0.9280,  0.0543,  0.2805,  0.1924, -0.2474, -1.2096, -1.7903],
        [ 2.6335,  0.7094,  1.4464, -0.4417,  0.2587,  1.4437, -0.2084,  0.4695],
        [-0.1834,  2.3721, -0.0296, -1.4014, -0.5290,  1.0046, -0.9236, -0.6688]])

In [75]:
# use query for embedding vector

query = input_weights[2]

#calculate attention scores w the snippet from in class use dot product
attention_scores2 = torch.empty(len(input_weights))
for i in range(len(input_weights)):
    attention_scores2[i] = torch.dot(query,input_weights[i])
print(attention_scores2)

tensor([ 4.8721,  0.3634, 12.1406,  2.9680])


In [77]:
#normalize using softmax function
attention_weights_2 = torch.softmax(attention_scores2, dim = 0)
attention_weights_2


tensor([6.9659e-04, 7.6711e-06, 9.9919e-01, 1.0376e-04])

In [78]:
#verify our sum = 1
attention_weights_2.sum()

tensor(1.)

In [83]:
# compute context vectors as the weighted sum with input weight vectors
context_vector_2 = torch.zeros (query.shape)
for i in range(len(attention_weights_2)):
    context_vector_2 += attention_weights_2[i] * input_weights[i]
context_vector_2

tensor([ 2.6322,  0.7082,  1.4453, -0.4416,  0.2577,  1.4432, -0.2088,  0.4706])

In [84]:
#get attention scores with matrix multiplication
attention_scores_2 = input_weights @ input_weights.T
attention_scores_2

tensor([[10.7647, -1.8986,  4.8721, -2.3657],
        [-1.8986,  6.2650,  0.3634, -0.7684],
        [ 4.8721,  0.3634, 12.1406,  2.9680],
        [-2.3657, -0.7684,  2.9680, 10.2148]])

In [92]:
#normalize again with softmax to find attention weights
attention_weights = torch.softmax(attention_scores_2, dim = -1)
attention_weights

tensor([[9.9724e-01, 3.1563e-06, 2.7521e-03, 1.9785e-06],
        [2.8373e-04, 9.9611e-01, 2.7244e-03, 8.7854e-04],
        [6.9659e-04, 7.6711e-06, 9.9919e-01, 1.0376e-04],
        [3.4359e-06, 1.6972e-05, 7.1190e-04, 9.9927e-01]])

In [94]:
#make sure the sum =1 
attention_weights[0].sum()

tensor(1.0000)

In [95]:
# now we calculate context vectors 
context_vectors = attention_weights @ input_weights
context_vectors

tensor([[ 1.2335, -1.1564,  0.0526, -0.0978, -0.9795,  0.9358, -0.6568,  2.3653],
        [ 0.7498, -0.9207,  0.0580,  0.2770,  0.1916, -0.2414, -1.2065, -1.7820],
        [ 2.6322,  0.7082,  1.4453, -0.4416,  0.2577,  1.4432, -0.2088,  0.4706],
        [-0.1814,  2.3709, -0.0285, -1.4007, -0.5285,  1.0049, -0.9231, -0.6680]])